In [16]:
import pandas as pd
import numpy as np

In [17]:
hotelreviews = pd.read_csv('raw_data/Hotel_Reviews.csv')

In [18]:
df = hotelreviews.copy()

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [20]:
df.head(3)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968


In [21]:
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


#Using preprocessing from previous assignment

stop_words = set(stopwords.words('english')) # you can also choose other languages

def preprocessing_stopwords(sentence):
    sentence = sentence.strip("") #whitespace
    sentence = sentence.lower() #lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) #remove number
    for punctuation in string.punctuation: #remove punc
        sentence = sentence.replace(punctuation, '')
    word_tokens = word_tokenize(sentence)
    
    stopwords_removed = [w for w in word_tokens if not w in stop_words] 
        # Lemmatizing the verbs
    verb_lemmatized = [                  
        WordNetLemmatizer().lemmatize(word, pos = "v") # v --> verbs
        for word in stopwords_removed]

    # 2 - Lemmatizing the nouns
    noun_lemmatized = [                 
        WordNetLemmatizer().lemmatize(word, pos = "n") # n --> nouns
        for word in verb_lemmatized]
    
    return noun_lemmatized


def preprocessing(sentence):
    sentence = sentence.strip("") #whitespace
    sentence = sentence.lower() #lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) #remove number
    for punctuation in string.punctuation: #remove punc
        sentence = sentence.replace(punctuation, '')
    word_tokens = word_tokenize(sentence)

        # Lemmatizing the verbs
    verb_lemmatized = [                  
        WordNetLemmatizer().lemmatize(word, pos = "v") # v --> verbs
        for word in word_tokens]

    # 2 - Lemmatizing the nouns
    noun_lemmatized = [                 
        WordNetLemmatizer().lemmatize(word, pos = "n") # n --> nouns
        for word in verb_lemmatized]
    
    #join_list = ' '.join(noun_lemmatized)
    
    return noun_lemmatized

In [22]:
#Testing, just use 'Hotel Arena' reviews only

# Create a boolean mask for 'Hotel_Name' equal to 'Hotel Arena'
# mask = df['Hotel_Name'] == 'Hotel Arena'

# Apply the mask to filter 'Hotel Arena' reviews only from the dataframe
# filtered_review = df[mask]

filtered_review = df.iloc[:200000,:]

In [23]:
#Applying preprocessing on the both positive and negative reviews

clean_negative_review = []
cleaned_positive_review = []


for sentence in filtered_review.Negative_Review:
    clean_negative_review.append(preprocessing_stopwords(sentence))

clean_negative_df = pd.DataFrame({'Review': clean_negative_review})    
clean_negative_df['Review'] = clean_negative_df['Review'].astype(str)


for sentence in filtered_review.Positive_Review:
    cleaned_positive_review.append(preprocessing_stopwords(sentence))
    
clean_positive_df = pd.DataFrame({'Review': cleaned_positive_review})    
clean_positive_df['Review'] = clean_positive_df['Review'].astype(str)

clean_negative_df.shape,clean_positive_df.shape

((200000, 1), (200000, 1))

In [24]:
#Applying preprocessing on the both positive and negative reviews

clean_negative_review = []
cleaned_positive_review = []


for sentence in filtered_review.Negative_Review:
    clean_negative_review.append(preprocessing(sentence))

clean_negative_df = pd.DataFrame({'Review': clean_negative_review})    
clean_negative_df['Review'] = clean_negative_df['Review'].astype(str)


for sentence in filtered_review.Positive_Review:
    cleaned_positive_review.append(preprocessing(sentence))
    
clean_positive_df = pd.DataFrame({'Review': cleaned_positive_review})    
clean_positive_df['Review'] = clean_positive_df['Review'].astype(str)

clean_negative_df.shape,clean_positive_df.shape

((200000, 1), (200000, 1))

In [25]:
# Get the number of rows in the DataFrame
num_negative_rows = clean_negative_df.shape[0]
num_positive_rows = clean_positive_df.shape[0]

# Create a new column with all 0 and 1 values based on positive/ negative
clean_negative_df['Label'] = [0] * num_negative_rows
clean_positive_df['Label'] = [1] * num_positive_rows

clean_negative_df.head(2),clean_positive_df.head(2)
#Combine both dataframe together to form dataset
combined_df = pd.concat([clean_negative_df, clean_positive_df], axis=0, join='inner')

combined_df['Label'].value_counts()

0    200000
1    200000
Name: Label, dtype: int64

In [26]:
from sklearn.model_selection import train_test_split
X = combined_df['Review']
y = combined_df['Label']

X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=True,test_size=0.3)

In [27]:
import gensim.downloader as api

word2vec_transfer = api.load("glove-wiki-gigaword-50")


In [28]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec_transfer, X_train)
X_test_embed = embedding(word2vec_transfer, X_test)

In [29]:
#embedding need max length of sentences so loop to find max length

max_length_negative = max(len(sublist) for sublist in clean_negative_df.Review)
max_length_positive = max(len(sublist) for sublist in clean_positive_df.Review)
max_length = max(max_length_negative,max_length_positive)
max_length,max_length_negative,max_length_positive

(3005, 3005, 2985)

In [30]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=max_length)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=max_length)

2023-06-21 11:41:37.363329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 11:41:37.563555: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-21 11:41:37.601765: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-21 11:41:37.601781: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

MemoryError: Unable to allocate 157. GiB for an array with shape (280000, 3005, 50) and data type float32

In [ ]:
#Baseline model

from sklearn.metrics import accuracy_score

unique, counts = np.unique(y_train, return_counts=True)
counts = dict(zip(unique, counts))
print('Number of labels in train set', counts)

y_pred = 0 if counts[0] > counts[1] else 1

print('Baseline accuracy: ', accuracy_score(y_test, [y_pred]*len(y_test)))

Number of labels in train set {0: 287, 1: 280}
Baseline accuracy:  0.48559670781893005


In [ ]:
#Creating a simple RNN model

from tensorflow.keras import Sequential
from tensorflow.keras import layers

def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

model = init_model()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train, 
          batch_size = 32,
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
13/13 [==============================] - 15s 883ms/step - loss: 0.6973 - accuracy: 0.4773 - val_loss: 0.7033 - val_accuracy: 0.4269
Epoch 2/100
13/13 [==============================] - 10s 809ms/step - loss: 0.6927 - accuracy: 0.5025 - val_loss: 0.6904 - val_accuracy: 0.5848
Epoch 3/100
13/13 [==============================] - 11s 818ms/step - loss: 0.6872 - accuracy: 0.5278 - val_loss: 0.6841 - val_accuracy: 0.5965
Epoch 4/100
13/13 [==============================] - 11s 839ms/step - loss: 0.6821 - accuracy: 0.5682 - val_loss: 0.7143 - val_accuracy: 0.4269
Epoch 5/100
13/13 [==============================] - 12s 894ms/step - loss: 0.6791 - accuracy: 0.5833 - val_loss: 0.6700 - val_accuracy: 0.6433
Epoch 6/100
13/13 [==============================] - 11s 874ms/step - loss: 0.6748 - accuracy: 0.5783 - val_loss: 0.6695 - val_accuracy: 0.6199
Epoch 7/100
13/13 [==============================] - 11s 851ms/step - loss: 0.6695 - accuracy: 0.6111 - val_loss: 0.6556 - val_accuracy:

In [ ]:
res = model.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 67.490%


## Trying 1D-CNN (Tokenized)

In [31]:
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


#Using preprocessing from previous assignment

stop_words = set(stopwords.words('english')) # you can also choose other languages

def preprocessing_stopwords_cnn(sentence):
    sentence = sentence.strip("") #whitespace
    sentence = sentence.lower() #lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) #remove number
    for punctuation in string.punctuation: #remove punc
        sentence = sentence.replace(punctuation, '')
    word_tokens = word_tokenize(sentence)
    
    stopwords_removed = [w for w in word_tokens if not w in stop_words] 
        # Lemmatizing the verbs
    verb_lemmatized = [                  
        WordNetLemmatizer().lemmatize(word, pos = "v") # v --> verbs
        for word in stopwords_removed]

    # 2 - Lemmatizing the nouns
    noun_lemmatized = [                 
        WordNetLemmatizer().lemmatize(word, pos = "n") # n --> nouns
        for word in verb_lemmatized]
    
    join_list = ' '.join(noun_lemmatized)
    
    return join_list


def preprocessing_cnn(sentence):
    sentence = sentence.strip("") #whitespace
    sentence = sentence.lower() #lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) #remove number
    for punctuation in string.punctuation: #remove punc
        sentence = sentence.replace(punctuation, '')
    word_tokens = word_tokenize(sentence)

        # Lemmatizing the verbs
    verb_lemmatized = [                  
        WordNetLemmatizer().lemmatize(word, pos = "v") # v --> verbs
        for word in word_tokens]

    # 2 - Lemmatizing the nouns
    noun_lemmatized = [                 
        WordNetLemmatizer().lemmatize(word, pos = "n") # n --> nouns
        for word in verb_lemmatized]
    
    join_list = ' '.join(noun_lemmatized)
    
    return join_list

In [32]:
#Applying preprocessing on the both positive and negative reviews

clean_negative_review = []
cleaned_positive_review = []


for sentence in filtered_review.Negative_Review:
    clean_negative_review.append(preprocessing_cnn(sentence))

clean_negative_df = pd.DataFrame({'Review': clean_negative_review})    
clean_negative_df['Review'] = clean_negative_df['Review'].astype(str)


for sentence in filtered_review.Positive_Review:
    cleaned_positive_review.append(preprocessing_cnn(sentence))
    
clean_positive_df = pd.DataFrame({'Review': cleaned_positive_review})    
clean_positive_df['Review'] = clean_positive_df['Review'].astype(str)

# Get the number of rows in the DataFrame
num_negative_rows = clean_negative_df.shape[0]
num_positive_rows = clean_positive_df.shape[0]

# Create a new column with all 0 and 1 values based on positive/ negative
clean_negative_df['Label'] = [0] * num_negative_rows
clean_positive_df['Label'] = [1] * num_positive_rows

clean_negative_df.head(2),clean_positive_df.head(2)
#Combine both dataframe together to form dataset
combined_df = pd.concat([clean_negative_df, clean_positive_df], axis=0, join='inner')

X = combined_df['Review']
y = combined_df['Label']

X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=True,test_size=0.3)

In [63]:
clean_negative_df.shape, clean_positive_df.shape, combined_df.shape

((200000, 2), (200000, 2), (400000, 2))

In [33]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


tk = Tokenizer()
tk.fit_on_texts(X_train)
X_train_token_cnn = tk.texts_to_sequences(X_train)
X_test_token_cnn = tk.texts_to_sequences(X_test)



In [34]:
max_train_cnn = max(len(list) for list in X_train_token_cnn)
max_test_cnn = max(len(list) for list in X_test_token_cnn)
max_length_cnn =max(max_train_cnn,max_test_cnn)


In [35]:
X_train_pad_cnn = pad_sequences(X_train_token_cnn, dtype='float32', padding='post',maxlen=max_length_cnn)
X_test_pad_cnn = pad_sequences(X_test_token_cnn, dtype='float32', padding='post',maxlen=max_length_cnn)

In [36]:
vocab_size = len(tk.word_index)
vocab_size

29636

In [38]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

# Size of your embedding space = size of the vector representing each word
embedding_size = 100

# Conv1D
model_cnn = Sequential()
model_cnn.add(layers.Embedding(
    input_dim=vocab_size+1, 
    input_length=max_length_cnn, # Max_sentence_length (optional, for model summary)
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer :)
))
model_cnn.add(layers.Conv1D(filters=20, kernel_size=3))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(10, activation="relu"))
model_cnn.add(layers.Dense(1, activation="sigmoid"))
model_cnn.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics='accuracy')

model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 392, 100)          2963700   
                                                                 
 conv1d (Conv1D)             (None, 390, 20)           6020      
                                                                 
 flatten (Flatten)           (None, 7800)              0         
                                                                 
 dense (Dense)               (None, 10)                78010     
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 3,047,741
Trainable params: 3,047,741
Non-trainable params: 0
_________________________________________________________________


2023-06-21 11:44:45.985170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-21 11:44:45.985377: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-21 11:44:45.985669: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-06-21 11:44:45.985735: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-06-21 11:44:45.985776: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [39]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn.fit(X_train_pad_cnn, y_train, 
          epochs=100, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
6125/6125 [==============================] - 41s 7ms/step - loss: 0.6932 - accuracy: 0.4992 - val_loss: 0.6932 - val_accuracy: 0.5008
Epoch 2/100
6125/6125 [==============================] - 38s 6ms/step - loss: 0.5246 - accuracy: 0.6440 - val_loss: 0.1510 - val_accuracy: 0.9464
Epoch 3/100
6125/6125 [==============================] - 37s 6ms/step - loss: 0.1482 - accuracy: 0.9481 - val_loss: 0.1519 - val_accuracy: 0.9456
Epoch 4/100
6125/6125 [==============================] - 38s 6ms/step - loss: 0.1422 - accuracy: 0.9509 - val_loss: 0.1468 - val_accuracy: 0.9513
Epoch 5/100
6125/6125 [==============================] - 39s 6ms/step - loss: 0.1406 - accuracy: 0.9515 - val_loss: 0.1429 - val_accuracy: 0.9522
Epoch 6/100
6125/6125 [==============================] - 38s 6ms/step - loss: 0.1381 - accuracy: 0.9522 - val_loss: 0.1489 - val_accuracy: 0.9510
Epoch 7/100
6125/6125 [==============================] - 38s 6ms/step - loss: 0.1375 - accuracy: 0.9531 - val_loss: 0.1426 -

In [40]:
res = model_cnn.evaluate(X_test_pad_cnn, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 95.148%


In [41]:
Good_test = "Great room and the view was amazing. The buffet breakfast had so many options and so much of everything as well. \
    The shops under the hotel were really good and also lots of little eating places too.\
        Raffles is a few mins walk and you can walk to Clarke Quay as well.\
            Perfect location and would highly recommend."
            
Bad_test = "the view was horrible. breakfast had so little options and so little of everything. \
    The shops under the hotel were really terrible and very little eating places too."
    
Good_test = preprocessing_cnn(Good_test)
Good_test_token = tk.texts_to_sequences(Good_test)
Good_test_pad = pad_sequences(Good_test_token, dtype='float32', padding='post',maxlen=max_length_cnn)
Bad_test = preprocessing_cnn(Bad_test)
Bad_test_token = tk.texts_to_sequences(Bad_test)
Bad_test_pad = pad_sequences(Bad_test_token, dtype='float32', padding='post',maxlen=max_length_cnn)

# Predict the sentiment of the good sentence
predictions_good = model_cnn.predict(Good_test_pad)
predicted_label_good = int(predictions_good[0][0] > 0.5)

# Predict the sentiment of the bad sentence
predictions_bad = model_cnn.predict(Bad_test_pad)
predicted_label_bad = int(predictions_bad[0][0] > 0.5)

predicted_label_good,predicted_label_bad


5/5 [==============================] - 0s 2ms/step


(0, 0)

In [64]:
predictions_good

array([[0.36721548],
       [0.44678435],
       [0.4112868 ],
       [0.32420632],
       [0.23397201],
       [0.4047879 ],
       [0.44678435],
       [0.71116257],
       [0.71116257],
       [0.34315896],
       [0.4047879 ],
       [0.32420632],
       [0.02103406],
       [0.43336147],
       [0.4047879 ],
       [0.23397201],
       [0.34924808],
       [0.4112868 ],
       [0.4047879 ],
       [0.6097729 ],
       [0.5382514 ],
       [0.4112868 ],
       [0.42350602],
       [0.4047879 ],
       [0.30724353],
       [0.4112868 ],
       [0.4047879 ],
       [0.32420632],
       [0.34315896],
       [0.32420632],
       [0.416127  ],
       [0.41128668],
       [0.4047879 ],
       [0.23397201],
       [0.34924808],
       [0.4112868 ],
       [0.4047879 ],
       [0.30724353],
       [0.49489826],
       [0.39102545],
       [0.39102545],
       [0.4112868 ],
       [0.23397201],
       [0.4047879 ],
       [0.30724353],
       [0.44678435],
       [0.4112868 ],
       [0.324

5/5 [==============================] - 0s 2ms/step


1

## Trying 1D-CNN (Word2Vec)

In [ ]:
max_train_cnn2 = max(len(list) for list in X_train)
max_test_cnn2 = max(len(list) for list in X_test)

max_length_cnn2 =max(max_train_cnn2,max_test_cnn2)

In [ ]:
# Embed the training and test sentences
X_train_embed_cnn2 = embedding(word2vec_transfer, X_train)
X_test_embed_cnn2 = embedding(word2vec_transfer, X_test)

# Pad the training and test embedded sentences
X_train_pad_cnn2 = pad_sequences(X_train_embed_cnn2, dtype='float32', padding='post', maxlen=max_length_cnn2)
X_test_pad_cnn2 = pad_sequences(X_test_embed_cnn2, dtype='float32', padding='post', maxlen=max_length_cnn2)

In [ ]:
# Conv1D
model_cnn2 = Sequential()
model_cnn2.add(layers.Conv1D(filters=20, kernel_size=3))
model_cnn2.add(layers.Flatten())
model_cnn2.add(layers.Dense(10, activation="relu"))
model_cnn2.add(layers.Dense(1, activation="sigmoid"))
model_cnn2.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics='accuracy')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn2.fit(X_train_pad_cnn2, y_train, 
          epochs=100, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
13/13 [==============================] - 1s 37ms/step - loss: 0.7516 - accuracy: 0.5631 - val_loss: 0.6468 - val_accuracy: 0.5556
Epoch 2/100
13/13 [==============================] - 0s 19ms/step - loss: 0.5998 - accuracy: 0.7020 - val_loss: 0.6656 - val_accuracy: 0.4971
Epoch 3/100
13/13 [==============================] - 0s 21ms/step - loss: 0.4637 - accuracy: 0.7753 - val_loss: 0.6689 - val_accuracy: 0.5322
Epoch 4/100
13/13 [==============================] - 0s 19ms/step - loss: 0.3283 - accuracy: 0.8712 - val_loss: 0.6994 - val_accuracy: 0.5731
Epoch 5/100
13/13 [==============================] - 0s 19ms/step - loss: 0.2327 - accuracy: 0.9419 - val_loss: 0.7568 - val_accuracy: 0.5965
Epoch 6/100
13/13 [==============================] - 0s 19ms/step - loss: 0.1771 - accuracy: 0.9470 - val_loss: 0.9530 - val_accuracy: 0.5380


In [ ]:
res = model_cnn2.evaluate(X_test_pad_cnn2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 58.848%


In [ ]:
Good_test = "Great room and the view was amazing. The buffet breakfast had so many options and so much of everything as well. \
    The shops under the hotel were really good and also lots of little eating places too.\
        Raffles is a few mins walk and you can walk to Clarke Quay as well.\
            Perfect location and would highly recommend."
            
Bad_test = "the view was horrible. breakfast had so little options and so little of everything. \
    The shops under the hotel were really terrible and very little eating places too."
    
Good_test = preprocessing_cnn(Good_test)
Good_test = embedding(word2vec_transfer, Good_test)
Good_test_pad = pad_sequences(Good_test, dtype='float32', padding='post',maxlen=max_length_cnn2)
Bad_test = preprocessing_cnn(Bad_test)
Bad_test = embedding(word2vec_transfer, Bad_test)
Bad_test_pad = pad_sequences(Bad_test, dtype='float32', padding='post',maxlen=max_length_cnn2)

# Predict the sentiment of the good sentence
predictions_good = model_cnn2.predict(Good_test_pad)
predicted_label_good = int(predictions_good[0][0] > 0.5)

# Predict the sentiment of the bad sentence
predictions_bad = model_cnn2.predict(Bad_test_pad)
predicted_label_bad = int(predictions_bad[0][0] > 0.5)

predicted_label_good,predicted_label_bad


5/5 [==============================] - 0s 3ms/step


(1, 1)

In [62]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from heapq import nlargest

stop_words = set(stopwords.words("english"))

# Convert set to list, remove 'again', and convert back to set
word_list = list(stop_words)
word_list.append('negative')
stop_words = set(word_list)

# Function to preprocess sentences
def preprocess(sentences):
    lemmatizer = WordNetLemmatizer()
    word_frequencies = defaultdict(int)

    for sentence in sentences:
        words = word_tokenize(sentence.lower())
        words = [lemmatizer.lemmatize(word) for word in words if word.isalnum()]
        
        for word in words:
            if word not in stop_words:
                word_frequencies[word] += 1

    return word_frequencies

# Function to extract keywords using TextRank
def extract_keywords(sentences, num_keywords=100):
    word_frequencies = preprocess(sentences)

    # Calculate weighted word frequencies
    max_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word] / max_frequency

    # Get top N keywords with highest scores
    top_keywords = nlargest(num_keywords, word_frequencies, key=word_frequencies.get)
    return top_keywords

# Extract keywords from the list of sentences
keywords = extract_keywords(X_train)

# Print the extracted keywords
print("Extracted Keywords:")
for keyword in keywords:
    print(keyword)


Extracted Keywords:
room
staff
location
hotel
breakfast
good
great
bed
friendly
clean
helpful
stay
small
nice
comfortable
excellent
would
service
get
bathroom
nothing
like
really
could
bar
lovely
night
u
one
positive
walk
close
restaurant
check
shower
make
time
everything
book
station
view
london
need
go
also
well
reception
work
even
price
little
bite
area
day
food
perfect
facility
quiet
comfy
floor
place
size
amaze
pay
park
ask
use
free
window
give
coffee
tube
take
next
door
water
much
back
love
quite
look
wifi
minute
poor
find
air
hot
expensive
come
welcome
upgrade
fantastic
extremely
lot
money
modern
leave
noisy
better
around
